In [ ]:
import livingpark_utils
import pandas as pd


utils = livingpark_utils.LivingParkUtils()

cohort = pd.read_csv("mak-etal-2017-7934773459255573745.csv")

###
from pathlib import Path
from livingpark_utils.dataset import ppmi
cohort["nifti_cache"] = cohort.apply(
            lambda row: ppmi.find_nifti_file_in_cache(
                row["PATNO"],
                row["EVENT_ID"],
                row["Description"],
                base_dir=Path("inputs").as_posix(),
            ),
            axis=1,
        )

#### TEST only
cohort = cohort[cohort["nifti_cache"] != ""].drop(columns=["nifti_cache"]).reset_index()
####

baseline = cohort[cohort["MRI_ID"].str.endswith("_Baseline")].sort_values(
            by=["PATNO"]
        )
follow_up = cohort[cohort["MRI_ID"].str.endswith("_Follow-up")].sort_values(
    by=["PATNO"]
)

In [ ]:
from livingpark_utils.pipeline.spm import SPM

spm = SPM(code_dir=utils.code_dir, cache=utils.data_cache_path)
spm.pairwise_registration(cohort, time_diff=1, force=False)

In [ ]:
baseline.apply(
    lambda subject:
    spm.spatial_normalization(subject, align_img_prefix="avg_", write_img_prefix="dv_"),
    axis=1
)

In [ ]:
spm.spatial_smoothing(baseline, img_prefix="wdv_")